# Tutorial for the Analysis of Sokoban Logfiles using Machine Learning (1)
# Read the data

## Basic Introduction

These documents introduces the detailed procedures of using machine learning techniques to analyze the logfiles of sokoban. Here is the first part, reading data.

Languages used include Python (3.0+) and R. I recommend you install [anaconda](https://www.continuum.io/downloads), a data science platform including all the tools and packages we need.

To write code in Python, the IDE Spyder is highly recommended. It provides a matlab-like interface, and also interactive programming experience, which is very handy in data analysis. Meanwhile, Pycharm, Visual Studio Code and sublime text are also very good IDEs.

For R, RStudio is the most renowned IDE. There is no better choice.

This tutorial is written in Jupyter Notebook, which allows the combination of documents and live codes.

All these tools can be installed by anaconda.

You may want to read tutorials for R and Python [here](). Besides the basics syntax of languages, you are also encouraged (or even required) to read documentations for packages such as scikit-learning in Python and data.table in R.

## Locating the logfiles

The first step (and the most tedious step) of the project is to read the data in the format of xml, and translate them into a format readable to Machine-Learning tools. To do so, let's locate all students' logfiles.

In [13]:
import os
from bs4 import BeautifulSoup
import os
import pickle
import sys
import numpy

cwd1 = "{}{}Grade 1-Sokoban+GL".format(os.getcwd(),os.sep)
folders = list(filter(lambda x: os.path.isdir(cwd1 + os.sep + x),
                      os.listdir(cwd1)))
folders = list(filter(lambda x: os.path.isfile(cwd1 + os.sep + x +
                                     os.sep + x + '02.xml'), folders))
print(folders[0:5], end = '...\n')
print('{} folders contains sokoban data in the folder Grade 1-Sokoban+GL'.format(len(folders)))
cwd2 = "{}{}Sokoban+Genetics".format(os.getcwd(),os.sep)
folders2 = list(filter(lambda x: os.path.isdir(cwd2 + os.sep + x),
                      os.listdir(cwd2)))
folders2 = list(filter(lambda x: os.path.isfile(cwd2 + os.sep + x +
                                     os.sep + x + '01.xml'), folders2))
print(folders2[0:5], end = '...\n')
print('{} folders contains sokoban data in the folder Sokoban+Genetics'.format(len(folders2)))
folders.extend(folders2)
print('{} folders in total.'.format(len(folders)))

['190101', '190102', '190103', '190104', '190105']...
219 folders contains sokoban data in the folder Grade 1-Sokoban+GL
['180102', '180103', '180104', '180105', '180106']...
159 folders contains sokoban data in the folder Sokoban+Genetics
378 folders in total.


## Preparation
### Build a dictionary to translate Chinese tags into English for further Parsing

In order to use BeautifulSoup to parse the xml file, we have to translate the Chinese tags into English.

Actually, the package xml.eTree.ElementTree can handle Chinese tags more readily without translation. Due to Time limitation, I have not implemented it in this file, while a sample code of using this package to read Sokoban logfile could be found [here](link.needed).

We have a translation dictionary "translate_dictionary.txt" containing the Chinese and corresponding English tags.

The first 9 lines of the dictionary are as follows:
>本关用时 guan_time  
>移动路径 path  
>路径时间 path_time  
>首次移动人物时间 first_move_time  
>首次移动箱子时间 first_push_time  
>本关结果 result  
>第1关 guan  
>第2关 guan  
>第3关 guan  
>...

In [14]:
transdic_content = open("translate_dictionary.txt", 'r', encoding='utf-8')
lines = transdic_content.read().splitlines()
dic = {}
for line in lines:
    key = line.split(' ')[0]
    val = line.split(' ')[1]
    dic[key] = val

### Create classes

Our idea is to use classes to do the reading-data work. Those classes can receive the logfiles in strings, parse them into variables, and finally return a formatted string containing those variables.

We define two hierarchical classes to process the data: Student and Guan.

In [15]:
from BFS_sokoban import visitedmap
'''
# This package is for solving the sokoban games. Function visitedmap can generate tracks of a subject's path, namely, 
a list of states (defined as the position of the person and boxes) from a given path, and the "guansolution_list" used 
in below is generated by another file ''solve_sokoban.py''. You do not need to fully understand how it works, as 
long as you know what I'm doing when I called them.
'''

class Student(object):
    
    def __init__(self, id, stu_soup, guansolution_list):
        '''
        The init method receives three arguments, id, which denotes the ID (the name of the folder) of a student;
        stu_soup, a bs4.BeautifulSoup object conataining the logfile data, generaged by BeautifulSoup, and 
        guansolution_list, consisting of tuples. Each tuple consists of 4 components, the level (ID of the guan), 
        the optimal path (a string), the number of steps, and the track (generated by the function vistedmap).
        '''
        self.id = id
        guan_soup_list = stu_soup.find_all('guan') #find_all method can extract all subnodes whose tags are 'guan'
        self.guan_list = [] #initialize a list to store data of each guan.
        for i, guan_soup in enumerate(guan_soup_list):
            if guan_soup.guan_time.string != '\n': # skip the guan where guan_time (本关用时) is empty ('\n' denotes newline)
                guan = Guan(i, guan_soup, guansolution_list[i]) # Use class Guan to process guan data.
                self.guan_list.append(guan)

    def export(self, varname):
        '''
        This method receives a list containing the variable names intended to be exported, and concatenate them into a 
        string.
        '''
        output_str = ''
        for guan in self.guan_list:
            guan_output_str = guan.guan_export(varname) # transfer variable name to guan_export method.
            output_str = output_str + self.id + guan_output_str
        return output_str

class Guan(object):
    def __init__(self, num, guan_soup, guansolution):
        self.export = {} #define the dictionary of variables to export.
        self.export["num"] = num+1 #since num starts from 0. Adding one will make the index more natural.
        self.export["guan_time"] = float(guan_soup.guan_time.string)
        first_move_time_str = guan_soup.first_move_time.string
        if len(first_move_time_str) > 1:
            self.export["first_move_time"] = float(first_move_time_str[0:-2])
        else:
            self.export["first_move_time"] = 'NA'
        first_push_time_str = guan_soup.first_push_time.string
        if len(first_push_time_str) > 1:
            self.export["first_push_time"] = float(first_push_time_str[0:-2])
        else:
            self.export['first_push_time'] = 'NA'
        self.result = guan_soup.result.string
        result_splited = self.result.split('：')
        if result_splited[0] == '正确':
            self.export["result_flag"] = 1
        if result_splited[0] == '步数过多':
            self.export["result_flag"] = 2
        if result_splited[0] == '放弃本关':
            self.export["result_flag"] = 3
        if result_splited[0] == '超时':
            self.export["result_flag"] = 4 # '步数过多' is labeled automatically by the software. I recommend 
            # merge it into '正确', as the criteria of '步数过多' is not explicit and somewhat misleading.
            # Subjects' performances are the same between the label "步数过多" and "正确".
        self.export["box_completed"] = int(result_splited[1][4])
        # Attention: Sometimes the format of the logfile will change, so number of boxes completed may not be 
        # the fourth charater. Please verify this index.
        # Sometimes the numbers of boxes completed even have not been recorded here by the software. In this 
        # situation, we have to calculate the number from the path. 
        
        self.path = []
        path_list = guan_soup.path.string.split(',')
        step_time_list = guan_soup.path_time.string.split('，')
        for direction, step_time in zip(path_list, step_time_list):
            if direction == "":
                continue
            push_flag = 0 # using push flag to denote whether this step is pushing the boxes or just moving.
            if len(direction) is 5:
                push_flag = 1 
                direction = direction[0]
            self.path.append([direction, push_flag, step_time])
        
        if self.path[0][0] == '\n':
            return None #The following analysis is based on the paths. If the paths are empty then skip this one.
        self.steptime = [float(x[2][:-1]) for x in self.path]
        # To send the path to the function visitedmap, we first tranfer the directions to a string.
        dic_letter = {'上': 'u', '下': 'd', '左': 'l', '右': 'r'}
        pathstr = ''
        for step in self.path:
            letter = dic_letter[step[0]]
            if step[1] == 1:
                letter = letter.upper() # Using upper characters to denote pushing and lower char moving.
            pathstr = pathstr + letter
        self.pathstr = pathstr
        level = guansolution[0]
        self.level = level
        self.track = visitedmap(level, pathstr) # calculate the subject's track of a Guan.
        same = 0 # Variable same counts the number of the states in the optimal path reached by the subject.
        for eachmap in guansolution[3]: 
            if eachmap in self.track:
                same += 1
        self.export['same_with_optimal'] = same
        duplicates = 0 # Variable duplicates counts the number of duplicated states in the subject's path.
        for eachmap in self.track:
            dup_num = self.track.count(eachmap)
            if dup_num > 1:
                duplicates += 1
            for i in range(dup_num):
                self.track.remove(eachmap) # Attention: A state will only be counted once, even if the subjects 
                # pass this states for many times. This is because some players may press left and right repeatedly
                # when they are thinking, which will create a great number of duplicated states. Actually, this 
                # behavior itself may reflect some traits of the subjects. It could be incorporated in our model 
                # in the future versions.
        optstep = guansolution[2]
        self.export['duplicates'] = duplicates
        self.export['steps'] = len(self.path)
        self.export['optstep'] = optstep
        self.export['dsteps'] = len(self.path) - optstep
        self.export['p_optimal'] = self.export['same_with_optimal'] / optstep
        # Deal with step time
        resttime = numpy.array(self.steptime[1:]) # remove the first-step time.
        if len(resttime) <= 3: 
            return None
        reststd = resttime.std() #calculate the standard deviation
        self.export['rest_sd'] = reststd
        restmean = resttime.mean()
        self.export['num_higher_than_sd'] = len([
            1 for x in resttime if x > restmean + reststd]) # numbers of steps which costing time higher than M+SD.
        # I use this to denote the thinking steps.
        self.export['rest_mean'] = numpy.array(
            [x for x in resttime if x <= restmean + reststd]).mean() # Attention: I re-calculate the mean again after
        # removing thinking steps. This mean can be used as the average excuting time.
        
    def guan_export(self, varname):
        guan_export_str = ''
        for var in varname:
            if var not in self.export:
                self.export[var] = 'NA'
            guan_export_str = guan_export_str + ',' + str(self.export[var])
        guan_export_str = guan_export_str + '\n'
        return guan_export_str

## Start Extraction

Every thing is ready. We can extract data now.

In [16]:
stulist = []
guansolution_list = pickle.load(open('guansolution_list.pdata', 'rb'))
for num, folder in enumerate(folders):
    if folder[1] == '9':            
        logfile = open(cwd1 + os.sep + folder + os.sep + folder
                   + '02.xml', "r", encoding='utf-8')
    else:
        logfile = open(cwd2 + os.sep + folder + os.sep + folder
                   + '01.xml', "r", encoding='utf-8')
    logfile_content = logfile.read()
    for key in dic: # using the dictionary to translate the tags into English.
        logfile_content = logfile_content.replace(key, dic[key])
    data_soup = BeautifulSoup(logfile_content, 'lxml') 
    student = Student(folder, data_soup, guansolution_list)
    stulist.append(student)


output = open('extracted_data.csv', 'w')

# Since not all instances of Guan contains all the variables (some of them may be NA), I use a tedious way to 
# extract the whole variable names.
# A smarter way is using Class variables, as I did in [here]().
max_len = 0
varname = []
for stu in stulist:
    temp = sorted(list(stu.guan_list[0].export.keys()))
    if max_len < len(temp):
        max_len = len(temp)
        varname = temp
output.write('id,' + ','.join(varname) + '\n') # write the variable name first.
for stu in stulist:
    output.write(stu.export(varname))
output.close()
print('Done')

Done
